# 任务一

In [1]:
import numpy as np
import pandas as pd
# 读取数据
mb = pd.read_excel('cumcm2018c1.xlsx') #会员信息表
sell = pd.read_csv('cumcm2018c2.csv') #商场销售流水表

/Users/quanzai/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
mb.head()

,kh,csrq,xb,djsj
0,c68b20b4,2002-11-02 00:00:00,0.0,2013-05-11 00:00:00.000
1,1ca15332,NaN,0.0,2004-11-04 16:31:52.436
2,a37cc182,1967-02-17 00:00:00,0.0,2004-12-31 21:24:34.216
3,2ab88539,1982-06-01 00:00:00,0.0,2010-11-19 00:00:00.000
4,b4c77269,1964-02-05 00:00:00,0.0,2007-12-14 00:00:00.000


In [4]:
sell.head()

,kh,dtime,spbm,sl,sj,je,spmc,jf,syjh,djh,gzbm,gzmc
0,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,290.0,270.20,兰芝化妆品正价瓶,270.20,6,25bb,8077.0,兰芝柜
1,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,325.0,302.80,兰芝化妆品正价瓶,302.80,6,25bb,8077.0,兰芝柜
2,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,195.0,181.80,兰芝化妆品正价瓶,181.80,6,25bb,8077.0,兰芝柜
3,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,270.0,251.55,兰芝化妆品正价瓶,251.55,6,25bb,8077.0,兰芝柜
4,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,2,245.0,456.55,兰芝化妆品正价瓶,456.55,6,25bb,8077.0,兰芝柜


## 重复值处理

In [2]:
#原始数据的大小
mb.shape
sell.shape

(194760, 4)

(1893532, 12)

In [3]:
mb = mb.drop_duplicates(subset=['kh']) #会员信息会员卡号去重
sell = sell.drop_duplicates() #商场销售流水表去重
mb.shape
sell.shape

(194754, 4)

(1893532, 12)

## 缺失值处理

In [4]:
mb.isnull().sum() #空数据检测
sell.isnull().sum()

kh          0
csrq    34187
xb       9435
djsj    12684
dtype: int64

kh       1017486
dtime          0
spbm           0
sl             0
sj             0
je             0
spmc           0
jf       1017486
syjh           0
djh            0
gzbm     1017486
gzmc     1036956
dtype: int64

数据一由于缺失的会员信息是固定的，无法预测，所以选择删除,
数据二的数据包含非会员的销售记录，缺少的卡号信息都是非会员消费信息

In [5]:
new_mb = mb.dropna() #去除空值后的数据
new_mb.isnull().sum()
new_mb.shape

kh      0
csrq    0
xb      0
djsj    0
dtype: int64

(153487, 4)

## 异常值处理

## 异常值检测

In [9]:
sell['dtime'] = pd.to_datetime(sell['dtime'])
mb['csrq'] = pd.to_datetime(mb['csrq'])
mb['djsj'] = pd.to_datetime(mb['djsj'])

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 9690-01-20 00:00:00

In [9]:
def over_date(date):
    try:
        pd.to_datetime(date)
        return True
    except:
        return False

In [10]:
csrq = mb['csrq'].apply(lambda x:over_date(x))
mb = mb[csrq]

转换日期类型后发现日期有异常，所以要对日期进行处理

In [11]:
sell['dtime'] = pd.to_datetime(sell['dtime'])
mb['csrq'] = pd.to_datetime(mb['csrq'])
mb['djsj'] = pd.to_datetime(mb['djsj'])

In [12]:
csrq = mb['csrq'].apply(lambda x:x.year >=1900 and x.year <= 2010)
mb = mb[csrq]

In [13]:
mb.shape

(143415, 4)

In [14]:
mb['csrq'].min() #出生年份最小值
mb['csrq'].max() #出生年份最大值
mb['djsj'].max() #入会时间最大值
mb['djsj'].min() #入会时间最小值

Timestamp('1900-01-01 00:00:00')

Timestamp('2010-12-04 00:00:00')

Timestamp('2080-07-03 00:00:00')

Timestamp('1900-01-01 00:00:00')

In [15]:
djsj = mb['djsj'].apply(lambda x:x.year >= 2000 and x.year <= 2021)
mb = mb[djsj]
mb.shape

(137324, 4)

In [20]:
sell.describe()

,sl,sj,je,jf,syjh,gzbm
count,1.893532e+06,1.893532e+06,1.893532e+06,876046.000000,1.893532e+06,876046.000000
mean,1.060906e+00,1.093785e+03,1.047361e+03,1035.463948,1.002693e+02,6328.248018
std,2.483580e+00,2.582808e+03,2.697462e+03,2064.978604,6.689405e+01,2371.230147
min,-3.810000e+02,1.000000e-01,-6.712575e+05,-201616.000000,1.000000e+00,0.000000
25%,1.000000e+00,3.200000e+02,3.100000e+02,180.000000,8.700000e+01,4334.000000
50%,1.000000e+00,5.700000e+02,5.640000e+02,510.000000,9.400000e+01,8064.000000
75%,1.000000e+00,1.100000e+03,1.098000e+03,1259.000000,1.010000e+02,8123.000000
max,1.492000e+03,1.342515e+06,1.342515e+06,268503.000000,3.200000e+02,9005.000000


In [16]:
sell['je'] = sell.je.apply(lambda x: 0 if x<0 else x) #负数金额改成0
sell['sl'] = sell.je.apply(lambda x: 0 if x<0 else x) #负数售价改成0
sell['jf'] = sell.je.apply(lambda x: 0 if x<0 else x) #负数积分改成0

In [24]:
sell.describe()

,sl,sj,je,jf,syjh,gzbm
count,1.893532e+06,1.893532e+06,1.893532e+06,1.893532e+06,1.893532e+06,876046.000000
mean,1.073132e+03,1.093785e+03,1.073132e+03,1.073132e+03,1.002693e+02,6328.248018
std,2.527245e+03,2.582808e+03,2.527245e+03,2.527245e+03,6.689405e+01,2371.230147
min,0.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,1.000000e+00,0.000000
25%,3.100000e+02,3.200000e+02,3.100000e+02,3.100000e+02,8.700000e+01,4334.000000
50%,5.640000e+02,5.700000e+02,5.640000e+02,5.640000e+02,9.400000e+01,8064.000000
75%,1.098000e+03,1.100000e+03,1.098000e+03,1.098000e+03,1.010000e+02,8123.000000
max,1.342515e+06,1.342515e+06,1.342515e+06,1.342515e+06,3.200000e+02,9005.000000


In [81]:
mb.head()

,kh,csrq,xb,djsj
0,c68b20b4,2002-11-02,0.0,2013-05-11 00:00:00.000
2,a37cc182,1967-02-17,0.0,2004-12-31 21:24:34.216
3,2ab88539,1982-06-01,0.0,2010-11-19 00:00:00.000
4,b4c77269,1964-02-05,0.0,2007-12-14 00:00:00.000
5,83a91070,1968-06-18,1.0,2004-05-01 16:50:13.950


In [25]:
task1 = pd.merge(mb,sell, how = 'outer',on = 'kh')

In [26]:
task1.head()

,kh,csrq,xb,djsj,dtime,spbm,sl,sj,je,spmc,jf,syjh,djh,gzbm,gzmc
0,c68b20b4,2002-11-02,0.0,2013-05-11 00:00:00.000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a37cc182,1967-02-17,0.0,2004-12-31 21:24:34.216,2016-09-25 10:49:14.016,d62a69e3,297.07,499.0,297.07,WMF D无,297.07,101.0,7cd8,7296.0,WMF 柜
2,a37cc182,1967-02-17,0.0,2004-12-31 21:24:34.216,2016-09-25 10:49:14.016,d62a69e3,11089.93,12288.0,11089.93,WMF D无,11089.93,101.0,7cd8,7296.0,WMF 柜
3,a37cc182,1967-02-17,0.0,2004-12-31 21:24:34.216,2017-02-28 17:17:35.533,252403ef,598.00,598.0,598.00,双立人商品 F无,598.00,102.0,ed0a,7242.0,双立人柜
4,2ab88539,1982-06-01,0.0,2010-11-19 00:00:00.000,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
task1.shape

(1999337, 15)

In [3]:
data = pd.read_csv('task1.csv')
data = data.dropna(axis = 0,subset = ['kh','djsj','dtime'])

/Users/quanzai/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,3,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data.to_csv('mb_sell.csv',index = False)